In [1]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import psycopg2
import threading
from datetime import datetime

DB_CONFIG = {
    "dbname": "lab2",
    "user": "postgres",
    "password": "password123",
    "host": "localhost",
    "port": "5432",
}

def setup_database():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    cur.execute("""
        CREATE TABLE IF NOT EXISTS user_counter (
            user_id SERIAL PRIMARY KEY,
            counter INTEGER NOT NULL DEFAULT 0,
            version INTEGER NOT NULL DEFAULT 0
        );
    """)
    
    cur.execute("""
        CREATE TABLE IF NOT EXISTS log_updates (
            id SERIAL PRIMARY KEY,
            method TEXT NOT NULL,
            start_time TIMESTAMP NOT NULL,
            end_time TIMESTAMP NOT NULL,
            total_updates INT NOT NULL
        );
    """)
    
    cur.execute("DELETE FROM user_counter WHERE user_id = 1;")
    cur.execute("INSERT INTO user_counter (user_id, counter, version) VALUES (1, 0, 0);")
    conn.commit()
    cur.close()
    conn.close()

def log_result(method, start_time, end_time, total_updates):
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    cur.execute("""
        INSERT INTO log_updates (method, start_time, end_time, total_updates)
        VALUES (%s, %s, %s, %s)
    """, (method, start_time, end_time, total_updates))
    conn.commit()
    cur.close()
    conn.close()

def get_final_counter():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    cur.execute("SELECT counter FROM user_counter WHERE user_id = 1;")
    final_value = cur.fetchone()[0]
    cur.close()
    conn.close()
    return final_value

def lost_update():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    for _ in range(10_000):
        cur.execute("SELECT counter FROM user_counter WHERE user_id = 1;")
        counter = cur.fetchone()[0] + 1
        cur.execute("UPDATE user_counter SET counter = %s WHERE user_id = 1;", (counter,))
        conn.commit()
    cur.close()
    conn.close()

def in_place_update():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    for _ in range(10_000):
        cur.execute("UPDATE user_counter SET counter = counter + 1 WHERE user_id = 1;")
        conn.commit()
    cur.close()
    conn.close()

def row_level_locking():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    for _ in range(10_000):
        cur.execute("SELECT counter FROM user_counter WHERE user_id = 1 FOR UPDATE;")
        counter = cur.fetchone()[0] + 1
        cur.execute("UPDATE user_counter SET counter = %s WHERE user_id = 1;", (counter,))
        conn.commit()
    cur.close()
    conn.close()

def optimistic_concurrency_control():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    for _ in range(10_000):
        while True:
            cur.execute("SELECT counter, version FROM user_counter WHERE user_id = 1;")
            counter, version = cur.fetchone()
            counter += 1
            cur.execute("UPDATE user_counter SET counter = %s, version = %s WHERE user_id = 1 AND version = %s;", 
                        (counter, version + 1, version))
            if cur.rowcount > 0:
                conn.commit()
                break
    cur.close()
    conn.close()

def run_test(method_name, method_func):
    print(f"Testing {method_name}...")
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    cur.execute("UPDATE user_counter SET counter = 0, version = 0 WHERE user_id = 1;")
    conn.commit()
    cur.close()
    conn.close()
    start_time = datetime.now()
    threads = [threading.Thread(target=method_func) for _ in range(10)]
    for thread in threads: thread.start()
    for thread in threads: thread.join()
    end_time = datetime.now()
    log_result(method_name, start_time, end_time, 10_000 * 10)
    print(f"{method_name} completed in {end_time - start_time}")
    print(f"Final counter value: {get_final_counter()}")

if __name__ == "__main__":
    setup_database()
    run_test("Lost-update", lost_update)
    run_test("In-place update", in_place_update)
    run_test("Row-level locking", row_level_locking)
    run_test("Optimistic concurrency control", optimistic_concurrency_control)

Testing Lost-update...
Lost-update completed in 0:00:21.385189
Final counter value: 10897
Testing In-place update...
In-place update completed in 0:00:17.989181
Final counter value: 100000
Testing Row-level locking...
Row-level locking completed in 0:00:30.660053
Final counter value: 100000
Testing Optimistic concurrency control...
Optimistic concurrency control completed in 0:00:42.100458
Final counter value: 100000
